<a href="https://colab.research.google.com/github/aboltCD/aboltCD/blob/main/Validaci%C3%B3n_Cruzada_%2B_Regresi%C3%B3n_Lineal_SGD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Validación Cruzada

En este notebook haremos predicciones sobre precios de casas en Boston utilizando regresiones lineales y el optimizador SGD.

Sin embargo, el plato fuerte esta vez es el uso de validación cruzada para estabilizar los resultados al disminuir los sesgos de seleccion de datos e hiperparámetros.

Comenzamos por importar las librerías necesarias:

In [88]:
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_boston
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd

Ahora cargamos el dataset de Boston Housing, lo separamos en Features y Target, lo escalamos y dividimos en entrenamiento y prueba (en este caso, utilizamos un 15% de los datos como prueba):

In [89]:
boston = load_boston()
#X, y = boston.data, boston.target

#print(boston.DESCR)

df = pd.DataFrame(boston.data, columns = boston.feature_names)
df["MEDV"] = boston.target

#Feature Matrix (todos los atributos)
X = df.drop("MEDV",1)  

#Feature Matrix (sólo la cantidad de habitaciones)
#X = df[["RM"]] 

#Target Variable
y = df[["MEDV"]]         

df.head()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.10, random_state = 1)

scalerX = StandardScaler().fit(X_train)
scalery = StandardScaler().fit(y_train)

X_train = scalerX.transform(X_train)
y_train = scalery.transform(y_train)
X_test = scalerX.transform(X_test)
y_test = scalery.transform(y_test)

y_train.shape


(455, 1)

Ahora entrenamos el modelo (ajustamos la regresión) y sacamos tres métricas de error: R^2, MSE (error cuadrado promedio), RMSE (raíz del error cuadrado promedio)

In [90]:
sgdr = SGDRegressor(learning_rate='invscaling', eta0=0.1, n_iter_no_change = 5)

#Métricas de validación cruzada para esa combinación de parámetros....
scores = cross_val_score(sgdr, X_train, y_train.ravel(), cv=10)

print("error promedio de validación cruzada", scores.mean())


error promedio de validación cruzada 0.6560687947237359


Sin embargo esta prueba y cálculo manual puede ser tedioso. Vamos a hacer algo más automático utilizando Grid Search Cross Validation para buscar el mejor parámetro en una "Grilla" de posibles candidatos (combinaciones de valores de parámetros) en donde cáda candidato será evaluado con validación cruzada: 

In [91]:
param_grid = [
  {'eta0': [0.2, 0.15, 0.1, 0.05, 0.01], 'learning_rate': ['optimal', 'constant', 'invscaling', 'adaptive'], 'n_iter_no_change': [1, 5, 10, 20, 50, 100]}
 ]


clf = GridSearchCV(estimator = SGDRegressor(), param_grid = param_grid, scoring = 'neg_mean_squared_error', cv=10) 
clf.fit(X_train, y_train.ravel())



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1187: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1187: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1187: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1187: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linea

GridSearchCV(cv=10, error_score=nan,
             estimator=SGDRegressor(alpha=0.0001, average=False,
                                    early_stopping=False, epsilon=0.1,
                                    eta0=0.01, fit_intercept=True,
                                    l1_ratio=0.15, learning_rate='invscaling',
                                    loss='squared_loss', max_iter=1000,
                                    n_iter_no_change=5, penalty='l2',
                                    power_t=0.25, random_state=None,
                                    shuffle=True, tol=0.001,
                                    validation_fraction=0.1, verbose=0,
                                    warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'eta0': [0.2, 0.15, 0.1, 0.05, 0.01],
                          'learning_rate': ['optimal', 'constant', 'invscaling',
                                            'adaptive'],
                          'n_iter_no

In [92]:


print("Mejor score (error minimo cuadrado negativo): ", clf.best_score_)
print("Mejores hiperparámetros: ", clf.best_params_)
print("Mejor modelo: ", clf.best_estimator_)


Mejor score (error minimo cuadrado):  -0.2843047852508456
Mejores hiperparámetros:  {'eta0': 0.15, 'learning_rate': 'invscaling', 'n_iter_no_change': 100}
Mejor modelo:  SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.15, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=1000,
             n_iter_no_change=100, penalty='l2', power_t=0.25,
             random_state=None, shuffle=True, tol=0.001,
             validation_fraction=0.1, verbose=0, warm_start=False)


Ahora podemos obtener los valores reales vs predecidos (no escalados) con la función StandardScaler.inverse_transform()